In [21]:
pip install dmba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 44.5 MB/s eta 0:00:00


In [34]:
import pandas as pd #data analysis
from sklearn.model_selection import train_test_split #machine learning
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

In [14]:
# 1 Read Tayko.csv!
tayko_data = pd.read_csv("Tayko.csv")

In [15]:
#Preprocess and clean the data
#processing can be done
# No need to clean the data
tayko_data.columns = [c.replace(' ', '_') for c in tayko_data.columns]
tayko_data.columns = [c.lower() for c in tayko_data.columns]
tayko_data


,address_us,freq,last_update,web,gender,address_res,purchase,spending
0,1,2,3662,1,0,1,1,128
1,1,0,2900,1,1,0,0,0
2,1,2,3883,0,0,0,1,127
3,1,1,829,0,1,0,0,0
4,1,1,869,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1995,1,1,1701,1,0,0,1,30
1996,1,1,2633,1,1,0,1,10
1997,1,0,3394,0,0,0,0,0
1998,1,1,253,0,1,1,0,0


In [13]:
tayko_data.loc[tayko_data['address_us'] == 0]

,address_us,freq,last_update,web,gender,address_res,purchase,spending
14,0,1,1465,0,0,1,1,192
15,0,1,2523,1,0,0,0,0
17,0,0,3694,1,0,0,0,0
26,0,0,3394,0,0,0,0,0
31,0,2,2947,1,0,0,1,352
...,...,...,...,...,...,...,...,...
1974,0,1,1708,0,0,0,1,98
1979,0,2,1710,0,0,1,0,0
1982,0,1,2620,1,0,0,0,0
1987,0,1,695,1,0,0,1,58


In [16]:
# 3 Partition your data into training and test sets, with the test set having 30% of the observations.
y = tayko_data["spending"]
X = tayko_data.loc[:,~tayko_data.columns.isin(['spending'])]
train_x,test_x,train_y,test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [18]:
# 4 Run a linear regression model with Spending as the dependent variable and all other variables as independent variables!
tayko_lm = LinearRegression()
print(train_x.shape)
print(train_y.shape)

tayko_lm.fit(train_x,train_y)


(1400, 7)
(1400,)


LinearRegression()

In [23]:
#5 Report the summary of the regression results on both training and test sets!
train_summary = regressionSummary(train_y, tayko_lm.predict(train_x))
test_summary = regressionSummary(test_y, tayko_lm.predict(test_x))




Regression statistics

               Mean Error (ME) : 0.0000
Root Mean Squared Error (RMSE) : 113.7214
     Mean Absolute Error (MAE) : 70.3883

Regression statistics

               Mean Error (ME) : 3.5798
Root Mean Squared Error (RMSE) : 141.7079
     Mean Absolute Error (MAE) : 81.1990


In [35]:
#6 Which variable is the most important one based on your result in the test set? How did you select that?
#print('intercept ', tayko_lm.intercept_)
#print(pd.DataFrame({'Predictor': test_x.columns, 'coefficient': tayko_lm.coef_}))
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_x[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_x[variables]), model)

best_model, best_variables = forward_selection(train_x.columns, train_model, score_model, verbose=True)

print(best_variables)
#Answer: ['freq', 'purchase', 'address_res', 'last_update'], from these last_update has lower AIC score , hence it is best.

Variables: address_us, freq, last_update, web, gender, address_res, purchase
Start: score=18535.28, constant
Step: score=17534.41, add freq
Step: score=17320.26, add purchase
Step: score=17249.38, add address_res
Step: score=17241.32, add last_update
Step: score=17241.32, add None
['freq', 'purchase', 'address_res', 'last_update']


In [41]:
#7 Please interpret the coefficient (in the test set) related to the most important variable that you found!
#Purchase   103.506197
print(pd.DataFrame({'Predictor': test_x.columns, 'coefficient': tayko_lm.coef_}))

# for one unit of increase in last_update, spendings increases by -0.009018 folds, given all other variables are constant.

     Predictor  coefficient
0   address_us    -6.545159
1         freq    76.072082
2  last_update    -0.009018
3          web    -1.963021
4       gender    -6.002036
5  address_res   -69.351897
6     purchase   103.506197
